In [1]:
from datasets import load_dataset

DATASET_NAME = "qasc"
# dataset = load_dataset("allenai/qasc", "", split=["train", "validation",'test'])
import pandas as pd
PARENT='QASC_Dataset_2Step'
train, dev, test=[pd.read_json(f'{PARENT}/{x}.jsonl', lines=True) for x in ['train','dev','test']]

In [2]:
def trimx(x):
    if len(x)>0 and x[-1]=='.':
        return x[:-1]
    return x
def get_choice_para(question_bundle):
    return [f"{'. '.join([trimx(m) for m in k['para'].split('. ')[-2:]])}| {k['text']}" for k in question_bundle['choices']]


In [3]:
choice_len = max([len(x['choices']) for x in train['question']])
choice_len

8

In [4]:
NUM_PERMUTE_QUESTION = None
EACH_HAS = 16
NUM_CHOICE=8

In [5]:
import pickle
from tqdm import tqdm, trange
import numpy as np


def get_prompt(dataset, index, location=-1, has_choice=False, dataset_index=0):
    is_test = dataset_index == 2
    try:
        json_line = dataset.loc[index]
        question = json_line["question"]
        choices = [x['text'] for x in question['choices']]
        choice_texts = get_choice_para(question)
        # print(choice_texts)
        if not is_test:
            answer_key = json_line["answerKey"]
            answer_key_idx = ord(answer_key[0]) - ord("A")
            answer_text = choices[answer_key_idx]
            if location > -1:
                perm = np.random.permutation(len(choice_texts)-1)
                choice_texts.remove(answer_text)
                choice_texts = [choice_texts[perm[i]].replace(
                    "\n", " ") for i in range(len(choice_texts))]
                choice_texts.insert(location, answer_text)
        if location == -1:
            perm = list(range(NUM_CHOICE))
            choice_texts = [choice_texts[perm[i]].replace(
                "\n", " ") for i in range(len(choice_texts))]
        candidates = " ".join(
            [
                f"( ) {text}" for text in choice_texts
            ]
        ).replace("\n", " ")

        # print(answer_key_idx,answer_text)
        prompt = f"{question['stem']} \\n {candidates}"
        if is_test:
            return (json_line['id'], prompt)
        else:
            return (prompt, answer_text)

    except:
        raise Exception(f'Answer key is {answer_key}')

        # print("QUESTION: ", question, "ANSWER KEY", answer_key)


if NUM_PERMUTE_QUESTION:
    container = []
    each_choice_has = EACH_HAS/NUM_CHOICE
    for ques_index in trange(NUM_PERMUTE_QUESTION):
        for idx in range(EACH_HAS):
            location = int(idx/each_choice_has)
            content = get_prompt(dataset, ques_index,
                                 location, has_choice=False)
            # print(content, location)
            container.append(content)

    pickle.dump(
        (NUM_PERMUTE_QUESTION, EACH_HAS, container),
        open(f"{DATASET_NAME}_test_permute.pkl", "wb"),
    )
else:
    for index, (dataset, dataname) in enumerate(zip([train, dev, test], ['train', 'dev', 'test'])):
        container = []
        for ques_index in trange(len(dataset)):
            prompt=get_prompt(dataset, ques_index,
                             has_choice=False, dataset_index=index)
            container.append(prompt)
        pickle.dump(container, open(f"{DATASET_NAME}_{dataname}.pkl", "wb"))

 11%|█         | 860/8134 [00:00<00:00, 8589.34it/s]

100%|██████████| 920/920 [00:00<00:00, 10802.07it/s]
